In [1]:
!pip install scikit-image
import numpy as np
from skimage.feature import hog
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd

from skimage.feature import greycomatrix, greycoprops

#Assignment

In [2]:
from scipy.stats import mode
 
#Euclidean Distance
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist

In [3]:
def predict(x_train, y , x_input, k):
    op_labels = []
     
    #Loop through the Datapoints to be classified
    for item in x_input: 
         
        #Array to store distances
        point_dist = []
         
        #Loop through each training Data
        for j in range(len(x_train)): 
            distances = eucledian(np.array(x_train[j,:]) , item) 
            #Calculating the distance
            point_dist.append(distances) 
        point_dist = np.array(point_dist) 
         
        #Sorting the array while preserving the index
        #Keeping the first K datapoints
        dist = np.argsort(point_dist)[:k] 
         
        #Labels of the K datapoints from above
        labels = y[dist]
         
        #Majority voting
        lab = mode(labels) 
        lab = lab.mode[0]
        op_labels.append(lab)
 
    return op_labels

In [4]:
!wget https://people.csail.mit.edu/celiu/CVPR2010/FMD/FMD.zip

--2022-04-06 10:02:28--  https://people.csail.mit.edu/celiu/CVPR2010/FMD/FMD.zip
Resolving people.csail.mit.edu (people.csail.mit.edu)... 128.30.2.133
Connecting to people.csail.mit.edu (people.csail.mit.edu)|128.30.2.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58627619 (56M) [application/zip]
Saving to: ‘FMD.zip’

FMD.zip             100%[===================>]  55.91M  11.5MB/s    in 4.9s    

2022-04-06 10:02:33 (11.5 MB/s) - ‘FMD.zip’ saved [58627619/58627619]



In [5]:
!unzip /content/FMD.zip

Archive:  /content/FMD.zip
   creating: mask/
   creating: mask/fabric/
  inflating: mask/fabric/fabric_moderate_001_new.jpg  
  inflating: mask/fabric/fabric_moderate_002_new.jpg  
  inflating: mask/fabric/fabric_moderate_003_new.jpg  
  inflating: mask/fabric/fabric_moderate_004_new.jpg  
  inflating: mask/fabric/fabric_moderate_005_new.jpg  
  inflating: mask/fabric/fabric_moderate_006_new.jpg  
  inflating: mask/fabric/fabric_moderate_007_new.jpg  
  inflating: mask/fabric/fabric_moderate_008_new.jpg  
  inflating: mask/fabric/fabric_moderate_009_new.jpg  
  inflating: mask/fabric/fabric_moderate_010_new.jpg  
  inflating: mask/fabric/fabric_moderate_011_new.jpg  
  inflating: mask/fabric/fabric_moderate_012_new.jpg  
  inflating: mask/fabric/fabric_moderate_013_new.jpg  
  inflating: mask/fabric/fabric_moderate_014_new.jpg  
  inflating: mask/fabric/fabric_moderate_015_new.jpg  
  inflating: mask/fabric/fabric_moderate_016_new.jpg  
  inflating: mask/fabric/fabric_moderate_017_new

In [6]:
import os
paths=[]
for dirname, _, filenames in os.walk('/content/image'):
    for filename in filenames:
        # print(dirname.split("/")[3])
        paths.append(os.path.join(dirname, filename))
len(paths)

1003

In [7]:
import cv2

In [81]:
def read_img(path):
    y=path.split('/')[3]
    x=cv2.imread(path,0)
    return x,y

In [82]:
X=[]
Y=[]
for path in paths:
    x,y=read_img(path)
    X.append(x)
    Y.append(y)

In [83]:

X[533]=X[534]
X[761]=X[762]
X[792]=X[793]
X[327]=X[328]
X[546]=X[547]
X[377]=X[378]


In [77]:
Y[546]

'foliage'

In [78]:
np.array(X[377]).shape

(384, 512)

In [84]:
temp=np.hstack(X)
Y=np.array(Y)

In [85]:
temp.shape

(384, 513536)

In [107]:
train=np.array(X)
train.shape

(1003, 384, 512)

In [108]:
def label(label):
    if label=='fabric':
        return 0
    elif label=='foliage':
        return 1
    elif label=='glass':
        return 2
    elif label=='leather':
        return 3
    elif label=='metal':
        return 4
    elif label=='paper':
        return 5
    elif label=='plastic':
        return 6
    elif label=='stone':
        return 7
    elif label=='water':
        return 8
    elif label=='wood':
        return 9
    else:
        print('<UNK>')
        return '<UNK>'

In [127]:
def GLCM(img,prop='contrast'):
    #x=[]
    #for i in range(3):
        glcm=greycomatrix(img, [1,2,3], [0,np.pi/4,np.pi/2])    
        #data=greycoprops(glcm)[0, 0]
        data1=greycoprops(glcm,'dissimilarity')[0, 0]
        data2=greycoprops(glcm,'homogeneity')[0, 0]
        data3=greycoprops(glcm,'ASM')[0, 0]
        data4=greycoprops(glcm,'energy')[0, 0]
        return(np.hstack((data1.flatten(),data2.flatten(),data2.flatten(),data3.flatten(),data4.flatten())))
        #return np.concatenate(x)


In [128]:
def GLCM_gray(img,prop='contrast'):
        glcm=greycomatrix(img, [1,3,5,7,9], [0,], )    
        data=greycoprops(glcm,prop)[0, 0]
        return data.flatten()

In [129]:
x_train=[]

for img in train:
    x_train.append(GLCM(img,'homogeneity'))
    

In [130]:
np.array(x_train).shape

(1003, 5)

In [131]:
x_train[50]

array([1.01637822e+01, 2.25394944e-01, 2.25394944e-01, 2.28851404e-04,
       1.51278354e-02])

In [132]:
y_label=[]
for lab in Y:
    y_label.append(label(lab))


In [133]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_label, test_size=0.3)

In [134]:
pred=predict(np.array(X_train),np.array(y_train),np.array(X_test),1)

In [135]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.15282392026578073

In [136]:
print(y_test)
print(pred)

[0, 8, 1, 8, 4, 9, 5, 5, 7, 0, 5, 5, 8, 3, 0, 6, 5, 8, 9, 2, 9, 1, 5, 6, 3, 6, 2, 4, 7, 2, 3, 4, 3, 9, 1, 1, 0, 0, 6, 6, 0, 0, 5, 7, 6, 2, 1, 6, 5, 4, 4, 2, 0, 6, 1, 6, 1, 3, 2, 0, 0, 7, 4, 3, 2, 0, 6, 6, 8, 5, 7, 7, 8, 0, 4, 2, 0, 8, 5, 0, 8, 4, 5, 6, 4, 8, 7, 6, 1, 6, 8, 3, 1, 0, 2, 7, 4, 1, 6, 8, 5, 2, 0, 5, 5, 7, 7, 4, 0, 6, 6, 0, 2, 4, 9, 6, 2, 2, 9, 9, 8, 9, 3, 8, 4, 1, 5, 2, 8, 8, 5, 5, 2, 9, 7, 6, 7, 0, 2, 4, 2, 9, 7, 2, 8, 2, 2, 2, 2, 8, 0, 0, 9, 0, 1, 8, 8, 1, 5, 8, 0, 9, 5, 3, 2, 7, 4, 0, 2, 8, 8, 7, 9, 6, 9, 1, 7, 9, 5, 9, 7, 4, 8, 0, 4, 6, 6, 5, 1, 8, 2, 3, 0, 7, 4, 7, 3, 1, 1, 8, 5, 3, 4, 3, 8, 4, 3, 4, 0, 4, 1, 4, 1, 9, 0, 3, 1, 5, 6, 4, 9, 0, 9, 1, 5, 7, 0, 9, 5, 1, 0, 4, 8, 2, 6, 7, 3, 1, 4, 7, 8, 5, 5, 5, 1, 1, 2, 8, 9, 8, 8, 1, 1, 6, 0, 2, 3, 3, 2, 8, 9, 8, 4, 6, 0, 7, 6, 3, 1, 9, 3, 4, 5, 2, 6, 9, 1, 8, 2, 3, 6, 9, 0, 8, 9, 9, 2, 3, 4, 2, 9, 8, 9, 6, 1, 5, 8, 5, 9, 2, 6]
[7, 9, 4, 3, 3, 7, 3, 4, 9, 0, 2, 2, 2, 9, 1, 6, 6, 8, 7, 0, 7, 7, 8, 5, 3, 5, 6, 5, 5, 4, 6, 1,

In [121]:
xx=[]
for img in X:
    xx.append(img.flatten())

In [122]:
pred=predict(np.array(xx),np.array(y_label),np.array(xx),1)

In [123]:
accuracy_score(np.array(y_label),pred)

1.0